# GR Comparison


In [24]:
import rebound
import reboundx
import time

In [38]:
def run(gr=None, swapMasses=False):  
    sim = rebound.Simulation()
    sim.integrator = "ias15"

    if swapMasses:
        sim.add(m=1.66013e-07,x=0.30749903826,vy=1.98009112946) # Mercury-like
        sim.add(m=1.) # Sun
    else:
        sim.add(m=1.) # Sun
        sim.add(m=1.66013e-07,x=0.30749903826,vy=1.98009112946) # Mercury-like
    sim.move_to_com() # Moves to the center of momentum frame

    if gr is not None:
        xs = reboundx.Extras(sim)
        methodToCall = getattr(xs, gr)
        methodToCall()

    sim.integrate(1000.)
    
    juliancentury = 628.33195 # in yr/2pi
    arcsec = 4.8481368e-06 # in rad
    if swapMasses:
        pomega = sim.particles[0].calculate_orbit(sim, primary=sim.particles[1]).pomega
    else:
        pomega = sim.particles[1].calculate_orbit(sim, primary=sim.particles[0]).pomega
    

    return pomega/sim.t*juliancentury/arcsec

In [40]:
modules = ["add_gr", "add_gr_potential", "add_gr_implicit"]

print("Ordered masses...")
for m in modules:
    start = time.time()
    res = run(m)
    end = time.time()
    print("%s:  \t%.10f   %.5fs"%(m,res,end-start))

print("\n\nSwapping masses...")
for m in modules:
    start = time.time()
    res = run(m,swapMasses=True)
    end = time.time()
    print("%s:  \t%.10f   %.5fs"%(m,res,end-start))

Ordered masses...
add_gr:  	43.0327387862   0.46940s
add_gr_potential:  	42.8805602912   0.36737s
add_gr_implicit:  	43.0327339760   1.74182s


Swapping masses...
add_gr:  	71.6198015059   0.45564s
add_gr_potential:  	0.0000000000   0.35604s
add_gr_implicit:  	43.0327339760   1.82993s
